# Compare Experiments

In [1]:
# load packages
import pickle
import numpy as np
from sklearn.svm import SVC

## Disaggregate correct/incorrect predictions

In [5]:
# read in Bool array yhat == ytrue
with open("y_match_bool", "rb") as f:
    y_match_bool = pickle.load(f)
    
# read in Bool array for unreliable tweets
with open("y_unrel", "rb") as f:
    is_unrel = pickle.load(f)

# read in Bool array for reliable tweets
with open("y_rel", "rb") as f:
    is_rel = pickle.load(f)
    
y_mismatch = (y_match_bool == False)

## Load list of explanations

In [6]:
# read in explanations list
with open("word_ica_netVals", "rb") as f:
    explanations = pickle.load(f)

In [7]:
explanations[0]

[('buy', 0.051218169268151266),
 ('today', 0.04564912805488858),
 ('recovered', 0.03684671343606235),
 ('coronavirus', 0.03167357242077659),
 ('2700', 0.030506518954913325),
 ('stocks', 0.029581742274224374),
 ('Reasons', 0.028743866428244537),
 ('deaths', 0.027860092852483344),
 ('virus', 0.025716989358801717),
 ('markets', 0.022864906695504512),
 ('1500', 0.02231262230616804),
 ('60', 0.021412463259315447),
 ('portfolio', 0.016436537908448373),
 ('back', 0.01619796505973802),
 ('new', 0.013312404914688995),
 ('cases', 0.01311502314673957),
 ('to', -0.01199561987952933)]

In [10]:
# separate by correctness of prediction
from itertools import compress
correctPred = list(compress(explanations, y_match_bool))
wrongPred = list(compress(explanations, y_mismatch))

In [11]:
wrongPred[0]

[('meth', 0.030987877447655538),
 ('your', 0.025280368484429324),
 ('for', 0.023607036528363858),
 ('This', 0.021277121458002424),
 ('with', 0.02115291971117502),
 ('coronavirus', 0.020399463946433768),
 ('contaminated', 0.020293806094854105),
 ('Florida', 0.01895198653657757),
 ('police', 0.01784072679125687),
 ('Is', 0.017506689596506714),
 ('will', 0.0156057483026649),
 ('dept', 0.011637419697194519),
 ('test', 0.011128987757850867),
 ('free', 0.005326991149549698),
 ('it', 0.002893041845161037)]

In [12]:
# get unique words from all explanations
with open('tweets_vocab_list', 'rb') as f:
    vocab_list = pickle.load(f)

len(vocab_list)

1659

In [13]:
# unique words from explanations for correctly predicted unreliable tweets
vocab_correct = []
for subList in correctPred:
    for el in subList:
        if el[0] not in vocab_correct:
            vocab_correct.append(el[0])
len(vocab_correct)

1530

In [14]:
# unique words from explanations for incorrectly predicted unreliable tweets
vocab_wrong = []
for subList in wrongPred:
    for el in subList:
        if el[0] not in vocab_wrong:
            vocab_wrong.append(el[0])
len(vocab_wrong)

271

## Experiment 1: Table 1 words

In [15]:
# define dictionary
t1 = [
    'blame',
    'accuse',
    'refuse',
    'catastrophe',
    'chaos',
    'evil',
    'fight',
    'danger',
    'hysteria',
    'panic',
    'paranoia',
    'laugh',
    'stupidity',
    'hear',
    'see',
    'feel',
    'suppose',
    'perceive',
    'look',
    'appear',
    'suggest',
    'believe',
    'pretend',
    'martial',
    'kill',
    'die',
    'weapon',
    'weaponizing',
    'ussr',
    'japan',
    'fukushima',
    'chernobyl',
    'wuhan',
    'china',
    'foreigners',
    'cats',
    'dogs',
    'i',
    'me',
    'mine',
    'my',
    'you',
    'your',
    'we',
    'our',
    'propaganda',
    'fake',
    'conspiracy',
    'claim',
    'misleading',
    'hoax',
    'cure',
    'breakthrough',
    'bitch',
    'wtf',
    'dogbreath',
    'zombie',
    'junkies',
    'hell',
    'screwed',
    'secular',
    'bible',
    'maga',
    'magat',
    'genetic',
    'hillary',
    'chinese',
    'fundamentalist',
    'market',
    'communist',
    'nazi',
    'stock',
    'economy',
    'money',
    'cost',
    'costs',
    'election',
    'campaign',
    'presidential',
    'impeachment',
    'rallies',
    'base',
    'trump',
    'war',
    'iran'
]

### Disaggregated by correctness of prediction

#### Correctly predicted

In [16]:
# get filtered vocab based on analysis words
filtered_vocab = [el for el in vocab_correct if el.lower() in t1]

In [17]:
# make dictionary from filtered vocab
myDict = dict()

for i in range(len(filtered_vocab)):
    myDict[filtered_vocab[i]] = i

In [18]:
# matrix of class associations of filtered vocab
my_matrix = np.zeros((len(correctPred), len(vocab_correct)))

for i in range(len(correctPred)):
    expl = correctPred[i]
    for j in range(len(expl)):
        word = expl[j][0]
        val = expl[j][1]
        
        if word in filtered_vocab:
            if val > 0:
                my_matrix[i, myDict[word]] = 1

            if val < 0:
                my_matrix[i, myDict[word]] = -1

In [19]:
# no penalty
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        if el > 0:
            rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [20]:
totalNonZero

206

In [21]:
np.array(tweet_scores).mean()

0.7320261437908496

In [22]:
# penalty for wrong class association of word
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [23]:
np.array(tweet_scores).mean()

0.7320261437908496

#### Incorrectly predicted

In [24]:
# get filtered vocab based on analysis words
filtered_vocab = [el for el in vocab_wrong if el.lower() in t1]

In [25]:
# make dictionary from filtered vocab
myDict = dict()

for i in range(len(filtered_vocab)):
    myDict[filtered_vocab[i]] = i

In [26]:
# matrix of class associations of filtered vocab
my_matrix = np.zeros((len(wrongPred), len(vocab_wrong)))

for i in range(len(wrongPred)):
    expl = wrongPred[i]
    for j in range(len(expl)):
        word = expl[j][0]
        val = expl[j][1]
        
        if word in filtered_vocab:
            if val > 0:
                my_matrix[i, myDict[word]] = 1

            if val < 0:
                my_matrix[i, myDict[word]] = -1

In [27]:
# no penalty
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        if el > 0:
            rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [28]:
np.array(tweet_scores).mean()

0.6296296296296297

In [29]:
# penalty for wrong class association of word
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [30]:
np.array(tweet_scores).mean()

0.6296296296296297

### Aggregated

In [31]:
# get filtered vocab based on analysis words
filtered_vocab = [el for el in vocab_list if el.lower() in t1]

In [32]:
filtered_vocab

['you',
 'we',
 'Trump',
 'You',
 'your',
 'look',
 'TRUMP',
 'We',
 'My',
 'fight',
 'evil',
 'Iran',
 'chaos',
 'refuse',
 'me',
 'Misleading',
 'junkies',
 'Zombie',
 'Market',
 'market',
 'stock',
 'Wuhan',
 'China',
 'conspiracy',
 'panic',
 'Stupidity',
 'Hell',
 'Paranoia',
 'weaponizing',
 'hoax',
 'campaign',
 'my',
 'I',
 'our',
 'rallies',
 'base',
 'screwed',
 'weapon',
 'Rallies',
 'laugh',
 'cure',
 'DIE',
 'Hoax',
 'economy',
 'Chinese',
 'see',
 'pretend',
 'suggest',
 'cost',
 'Communist',
 'kill',
 'hysteria',
 'Fake',
 'war',
 'impeachment',
 'Conspiracy',
 'Cure',
 'FAKE',
 'fake',
 'ECONOMY',
 'MARKET',
 'STOCK',
 'money',
 'Chernobyl',
 'USSR',
 'Fukushima',
 'Japan',
 'KILL',
 'MY',
 'feel',
 'believe',
 'MAGA',
 'Propaganda']

In [33]:
len(filtered_vocab)

73

In [34]:
# make dictionary from filtered vocab
myDict = dict()

for i in range(len(filtered_vocab)):
    myDict[filtered_vocab[i]] = i

In [35]:
# matrix of class associations of filtered vocab
my_matrix = np.zeros((len(explanations), len(vocab_list)))

for i in range(len(explanations)):
    expl = explanations[i]
    for j in range(len(expl)):
        word = expl[j][0]
        val = expl[j][1]
        
        if word in filtered_vocab:
            if val > 0:
                my_matrix[i, myDict[word]] = 1

            if val < 0:
                my_matrix[i, myDict[word]] = -1

In [36]:
# no penalty
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        if el > 0:
            rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [37]:
totalNonZero

232

In [38]:
np.array(tweet_scores).mean()

0.7166666666666667

In [39]:
# penalty for wrong class association of word
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [40]:
np.array(tweet_scores).mean()

0.7166666666666667

## Experiment 2:  Table 1 + manual additions

In [41]:
# define dictionary
t1PlusManual = [
    'blame',
    'accuse',
    'refuse',
    'catastrophe',
    'emergency',
    'chaos',
    'crisis',
    'evil',
    'fight',
    'danger',
    'hysteria',
    'panic',
    'paranoia',
    'fear',
    'fears',
    'laugh',
    'stupidity',
    'hear',
    'see',
    'feel',
    'suppose',
    'perceive',
    'look',
    'appear',
    'suggest',
    'believe',
    'believed',
    'pretend',
    'martial',
    'kill',
    'killing',
    'kills',
    'killed',
    'die',
    'death',
    'dies',
    'dying',
    'dead',
    'died',
    'threat',
    'weapon',
    'weaponize',
    'weaponizing',
    'knife',
    'ussr',
    'japan',
    'chernobyl',
    'wuhan',
    'china',
    'foreigners',
    'cat',
    'cats',
    'dog',
    'dogs',
    'i',
    'me',
    'mine',
    'my',
    'you',
    'yours',
    'your',
    'we',
    'our',
    'propaganda',
    'fake',
    'conspiracy',
    'claim',
    'claims',
    'claiming',
    'claimed',
    'misleading',
    'hoax',
    'cure',
    'breakthrough',
    'bitch',
    'wtf',
    'dogbreath',
    'zombie',
    'junkies',
    'hell',
    'screwed',
    'fuck',
    'fucking',
    'fucked',
    'fuckin',
    'wth',
    'secular',
    'bible',
    'maga',
    'magat',
    'genetic',
    'hillary',
    'clinton',
    'fundamentalist',
    'market',
    'communist',
    'nazi',
    'stock',
    'bank',
    'economy',
    'economic',
    'money',
    'cost',
    'costs',
    'election',
    'campaign',
    'presidential',
    'impeachment',
    'rally',
    'rallies',
    'base',
    'president',
    'trump',
    'war',
    'wwiii',
    'asteroid',
    'banknotes',
    'dangerous',
    'invent',
    'invented',
    'iran',
    'lie',
    'lies',
    'lying',
    'lied',
    'liar',
    'liars',
    'lmfao',
    'lmfaoooooo',
    'misinformation',
    'news',
    'media',
    'financial',
    'propagandawars',
    'antidote'
]

### Disaggregated by correctness of prediction

#### Correctly predicted

In [42]:
# get filtered vocab based on analysis words
filtered_vocab = [el for el in vocab_correct if el.lower() in t1PlusManual]

In [43]:
# make dictionary from filtered vocab
myDict = dict()

for i in range(len(filtered_vocab)):
    myDict[filtered_vocab[i]] = i

In [44]:
# matrix of class associations of filtered vocab
my_matrix = np.zeros((len(correctPred), len(vocab_correct)))

for i in range(len(correctPred)):
    expl = correctPred[i]
    for j in range(len(expl)):
        word = expl[j][0]
        val = expl[j][1]
        
        if word in filtered_vocab:
            if val > 0:
                my_matrix[i, myDict[word]] = 1

            if val < 0:
                my_matrix[i, myDict[word]] = -1

In [45]:
# no penalty
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        if el > 0:
            rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [46]:
totalNonZero

301

In [47]:
np.array(tweet_scores).mean()

0.8235294117647058

In [48]:
# penalty for wrong class association of word
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [49]:
np.array(tweet_scores).mean()

0.8235294117647058

#### Incorrectly predicted

In [50]:
# get filtered vocab based on analysis words
filtered_vocab = [el for el in vocab_wrong if el.lower() in t1PlusManual]

In [51]:
# make dictionary from filtered vocab
myDict = dict()

for i in range(len(filtered_vocab)):
    myDict[filtered_vocab[i]] = i

In [52]:
# matrix of class associations of filtered vocab
my_matrix = np.zeros((len(wrongPred), len(vocab_wrong)))

for i in range(len(wrongPred)):
    expl = wrongPred[i]
    for j in range(len(expl)):
        word = expl[j][0]
        val = expl[j][1]
        
        if word in filtered_vocab:
            if val > 0:
                my_matrix[i, myDict[word]] = 1

            if val < 0:
                my_matrix[i, myDict[word]] = -1

In [53]:
# no penalty
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        if el > 0:
            rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [54]:
np.array(tweet_scores).mean()

0.7777777777777778

In [55]:
# penalty for wrong class association of word
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [56]:
np.array(tweet_scores).mean()

0.7777777777777778

### Aggregated

In [57]:
# get filtered vocab based on analysis words
filtered_vocab = [el for el in vocab_list if el.lower() in t1PlusManual]

In [58]:
filtered_vocab

['you',
 'we',
 'President',
 'Trump',
 'You',
 'your',
 'look',
 'media',
 'TRUMP',
 'We',
 'WWIII',
 'dying',
 'asteroid',
 'BANKNOTES',
 'dangerous',
 'economic',
 'My',
 'believed',
 'knife',
 'fight',
 'NEWS',
 'EMERGENCY',
 'evil',
 'invented',
 'Iran',
 'chaos',
 'refuse',
 'lying',
 'me',
 'Misleading',
 'junkies',
 'Zombie',
 'Market',
 'market',
 'stock',
 'Wuhan',
 'claims',
 'China',
 'conspiracy',
 'killing',
 'LMFAOOOOOO',
 'lied',
 'threat',
 'crisis',
 'Bank',
 'panic',
 'Stupidity',
 'Hell',
 'Death',
 'died',
 'Claims',
 'Fears',
 'Paranoia',
 'weaponizing',
 'hoax',
 'claiming',
 'Fear',
 'campaign',
 'Kills',
 'my',
 'death',
 'I',
 'fucked',
 'our',
 'rallies',
 'base',
 'screwed',
 'emergency',
 'weapon',
 'Rallies',
 'Clinton',
 'DEAD',
 'laugh',
 'cure',
 'DIE',
 'lie',
 'Hoax',
 'lies',
 'economy',
 'dog',
 'see',
 'news',
 'pretend',
 'misinformation',
 'suggest',
 'cost',
 'Communist',
 'fuckin',
 'kill',
 'fear',
 'financial',
 'fucking',
 'hysteria',
 'kill

In [59]:
len(filtered_vocab)

125

In [60]:
# make dictionary from filtered vocab
myDict = dict()

for i in range(len(filtered_vocab)):
    myDict[filtered_vocab[i]] = i

In [61]:
# matrix of class associations of filtered vocab
my_matrix = np.zeros((len(explanations), len(vocab_list)))

for i in range(len(explanations)):
    expl = explanations[i]
    for j in range(len(expl)):
        word = expl[j][0]
        val = expl[j][1]
        
        if word in filtered_vocab:
            if val > 0:
                my_matrix[i, myDict[word]] = 1

            if val < 0:
                my_matrix[i, myDict[word]] = -1

In [62]:
# no penalty
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        if el > 0:
            rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [63]:
totalNonZero

341

In [64]:
np.array(tweet_scores).mean()

0.8166666666666667

In [65]:
# penalty for wrong class association of word
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [66]:
np.array(tweet_scores).mean()

0.8166666666666667

## Experiment 3: Table 1 words + stemming

In [67]:
from nltk.stem.porter import *

In [120]:
stemmer = PorterStemmer()

In [121]:
# define dictionary
stems = [stemmer.stem(word) for word in t1]
stems = list(set(stems))

In [122]:
stems

['catastroph',
 'appear',
 'mine',
 'look',
 'wuhan',
 'elect',
 'our',
 'china',
 'conspiraci',
 'bitch',
 'refus',
 'campaign',
 'suppos',
 'perceiv',
 'die',
 'feel',
 'believ',
 'i',
 'hoax',
 'evil',
 'cat',
 'accus',
 'ralli',
 'iran',
 'magat',
 'dog',
 'your',
 'ussr',
 'market',
 'my',
 'fake',
 'me',
 'hell',
 'secular',
 'cure',
 'wtf',
 'blame',
 'pretend',
 'fukushima',
 'nazi',
 'genet',
 'we',
 'chines',
 'chernobyl',
 'impeach',
 'you',
 'breakthrough',
 'trump',
 'suggest',
 'zombi',
 'presidenti',
 'chao',
 'kill',
 'hysteria',
 'danger',
 'hear',
 'base',
 'see',
 'mislead',
 'fight',
 'war',
 'money',
 'screw',
 'communist',
 'hillari',
 'martial',
 'maga',
 'cost',
 'panic',
 'paranoia',
 'propaganda',
 'junki',
 'economi',
 'laugh',
 'claim',
 'stock',
 'dogbreath',
 'bibl',
 'japan',
 'fundamentalist',
 'foreign',
 'weapon',
 'stupid']

### Disaggregated by correctness of prediction

#### Correctly predicted

In [123]:
# get filtered vocab based on analysis words
filtered_vocab = [el for el in vocab_correct if stemmer.stem(el).lower() in stems]

In [124]:
# make dictionary from filtered vocab
myDict = dict()

for i in range(len(filtered_vocab)):
    myDict[filtered_vocab[i]] = i

In [125]:
# matrix of class associations of filtered vocab
my_matrix = np.zeros((len(correctPred), len(vocab_correct)))

for i in range(len(correctPred)):
    expl = correctPred[i]
    for j in range(len(expl)):
        word = expl[j][0]
        val = expl[j][1]
        
        if word in filtered_vocab:
            if val > 0:
                my_matrix[i, myDict[word]] = 1

            if val < 0:
                my_matrix[i, myDict[word]] = -1

In [126]:
# no penalty
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        if el > 0:
            rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [127]:
totalNonZero

248

In [128]:
np.array(tweet_scores).mean()

0.7908496732026143

In [129]:
# penalty for wrong class association of word
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [130]:
np.array(tweet_scores).mean()

0.7908496732026143

#### Incorrectly predicted

In [131]:
# get filtered vocab based on analysis words
filtered_vocab = [el for el in vocab_wrong if stemmer.stem(el).lower() in stems]

In [132]:
# make dictionary from filtered vocab
myDict = dict()

for i in range(len(filtered_vocab)):
    myDict[filtered_vocab[i]] = i

In [133]:
# matrix of class associations of filtered vocab
my_matrix = np.zeros((len(wrongPred), len(vocab_wrong)))

for i in range(len(wrongPred)):
    expl = wrongPred[i]
    for j in range(len(expl)):
        word = expl[j][0]
        val = expl[j][1]
        
        if word in filtered_vocab:
            if val > 0:
                my_matrix[i, myDict[word]] = 1

            if val < 0:
                my_matrix[i, myDict[word]] = -1

In [134]:
# no penalty
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        if el > 0:
            rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [135]:
np.array(tweet_scores).mean()

0.7037037037037037

In [136]:
# penalty for wrong class association of word
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [137]:
np.array(tweet_scores).mean()

0.7037037037037037

### Aggregated

In [138]:
# get filtered vocab based on analysis words
filtered_vocab = [el for el in vocab_list if stemmer.stem(el).lower() in stems]

In [139]:
filtered_vocab

['stocks',
 'markets',
 'you',
 'we',
 'Trump',
 'You',
 'your',
 'look',
 'TRUMP',
 'We',
 'dying',
 'dangerous',
 'suggests',
 'My',
 'believed',
 'refused',
 'fight',
 'evil',
 'Iran',
 'chaos',
 'refuse',
 'me',
 'Misleading',
 'junkies',
 'Zombie',
 'Fundamentalists',
 'Market',
 'market',
 'stock',
 'Wuhan',
 'claims',
 'China',
 'conspiracy',
 'killing',
 'panic',
 'Stupidity',
 'Hell',
 'died',
 'Impeach',
 'Claims',
 'Paranoia',
 'weaponizing',
 'hoax',
 'claiming',
 'campaign',
 'Kills',
 'my',
 'I',
 'refuses',
 'our',
 'rallies',
 'base',
 'screwed',
 'weapon',
 'Rallies',
 'laugh',
 'cure',
 'DIE',
 'Hoax',
 'economy',
 'Chinese',
 'dog',
 'see',
 'pretend',
 'foreign',
 'suggest',
 'cost',
 'looks',
 'Communist',
 'kill',
 'Genetically',
 'hysteria',
 'seeing',
 'kills',
 'killed',
 'Fake',
 'Nazis',
 'war',
 'impeachment',
 'Killed',
 'conspiracies',
 'Conspiracy',
 'Cure',
 'FAKE',
 'fake',
 'ECONOMY',
 'MARKET',
 'STOCK',
 'elections',
 'Appearing',
 'money',
 'Chernob

In [140]:
len(filtered_vocab)

103

In [141]:
# make dictionary from filtered vocab
myDict = dict()

for i in range(len(filtered_vocab)):
    myDict[filtered_vocab[i]] = i

In [142]:
# matrix of class associations of filtered vocab
my_matrix = np.zeros((len(explanations), len(vocab_list)))

for i in range(len(explanations)):
    expl = explanations[i]
    for j in range(len(expl)):
        word = expl[j][0]
        val = expl[j][1]
        
        if word in filtered_vocab:
            if val > 0:
                my_matrix[i, myDict[word]] = 1

            if val < 0:
                my_matrix[i, myDict[word]] = -1

In [143]:
# no penalty
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        if el > 0:
            rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [144]:
totalNonZero

276

In [157]:
explanations[4]

[('the', 0.18333344931271334),
 ('unironic', 0.08000315840922398),
 ('NYT', 0.0628706663429014),
 ('Israelis', 0.0552038627822382),
 ('to', 0.04690367574080995),
 ('a', 0.04602305949756075),
 ('an', 0.045759952393576754),
 ('for', 0.04431488083685313),
 ('is', 0.04101546354827027),
 ('write', 0.03824383824026996),
 ('coronavirus', 0.038199697093392104),
 ('100', 0.03639169740551942),
 ('do', 0.03616979879774638),
 ('finding', 0.034518224116837234),
 ('likely', 0.030694457454544192),
 ('up', 0.03027192413811323),
 ('end', 0.026369508571869902),
 ('If', 0.026111692974492583),
 ('roughly', 0.024855474977400767),
 ('vaccine', 0.020854525884523082)]

In [145]:
np.array(tweet_scores).mean()

0.7777777777777778

In [146]:
# penalty for wrong class association of word
tweet_scores = []
totalNonZero = 0
for row in my_matrix:
    rowSum = 0
    nonZero = 0
    for el in row:
        rowSum += el
        if el != 0:
            nonZero += 1
            totalNonZero += 1
    
    if nonZero > 0:
        score = rowSum/nonZero
    else:
        score = 0
    
    tweet_scores.append(score)

In [147]:
np.array(tweet_scores).mean()

0.7777777777777778